In [156]:
"""
Plot voltages from a model_run.py output file
"""
%matplotlib qt
import os
import matplotlib
matplotlib.use('Qt5Agg')
import pickle
import numpy as np
from cnmodel.util import sound
import matplotlib.pyplot as mpl
from pathlib import Path
import pylibrary.PlotHelpers as PH
from cnmodel.util import vector_strength
import json
from vcnmodel import cell_config

freq = 4000.
db = 40
SR = 'MS'
nreps = 20
fmod = 400.
dmod = 50

cconfig = cell_config.CellConfig()

titlestring = f"AN PSTH {freq/1000:.1f}kHz, {int(db):2d}dbSPL, {SR:s} fibers, {nreps:d} reps, SAM {int(fmod):d}/{dmod:d}% AN SPikes"

def get_sims(cell, inp):
    
    scf = cconfig.get_soma_ratio(cell)
    # fnb = 'VCN_Cells/VCN_c09/Simulations/AN/AN_Result_VCN_c09_inp=self'
    # fnb + '_XM13nacncoop_II_soma=1.489_dend=1.236_Syn000_multisite_001_tonepip_030dB_16000.0_HS.p'
    basedir = '/Users/pbmanis/Desktop/Python/VCNModel'
    fnb = f'VCN_Cells/{cell:s}/Simulations/AN/AN_Result_{cell:s}_inp={inp:s}'  # or input = 'VCN_c02' etc
    fnb += f'_XM13_nacncoop_II_soma={scf:.3f}_dend={scf:.3f}_ASA={scf:.3f}_delays_multisite_{nreps:03d}_SAM_{db:03d}dB_{freq:.1f}_{fmod:.1f}_{int(dmod):03d}_{SR:2s}.p'
    return [Path(basedir, fnb)]
    print('fnb: ', fnb)
    matchstr = f"SAM_{db:03d}dB_{freq:.1f}_{fmod:.1f}_{int(dmod):03d})"
    print('matchstr: ', matchstr)
    print('match: ', fnb.find(matchstr))
    fnsyn = Path(basedir, fnb[:(fnb.find(matchstr)+len(matchstr))])
    print('fnsyn: ', fnsyn)
    fns = fnsyn.parent.glob(str(fnsyn.name)+'*.p')
    print('fns: ', list(fns))
    fnsl0 = sorted(list(fns))
    fnsl = []
    print('fnsl0: ', fnsl0)
    for i in range(len(fnsl0)):
        if str(fnsl0[i]).find(f'_{nreps:03d}_SAM') > 0 and str(fnsl0[i]).find(f'_4000.0') > 0:
            fnsl.append(fnsl0[i])
    nax = len(fnsl)
    for f in fnsl:
        print('name: ', str(f.name))
    return fnsl

# # fnb = 'ForGeorge/AN_Result_VCN_c09_inp=self_XM13nacncoop_II_soma=1.489_dend=1.236_delays_multisite_001_tonepip_030dB_16000.0_HS.p'
# # fnbo = 'ForGeorge/AN_Result_VCN_c09_inp=self_XM13nacncoop_II_soma=1.489_dend=1.236_delays_multisite_001_tonepip_030dB_16000.0_HS_noPosixPath.p'
# # VCN_Cells/VCN_c09/Simulations/AN/AN_Result_VCN_c09_inp=self_XM13nacncoop_II_soma=1.489_dend=1.236_delays_multisite_050_SAM_030dB_16000.0_200.0_100_MS.p

# fnb = "VCN_Cells/VCN_c09/Simulations/AN/AN_Result_VCN_c09_inp=self_XM13nacncoop_II_soma=1.489_dend=1.236_delays_multisite_050_SAM_030dB_16000.0_200.0_100_MS.p"

# fnb2 = "VCN_Cells/VCN_c09/Simulations/AN/AN_Result_VCN_c09_inp=VCN_c10_XM13nacncoop_II_soma=1.489_dend=1.236_delays_multisite_050_SAM_030dB_16000.0_200.0_100_MS.p"
# print(os.getcwd())

# if not Path(fnb).is_file():
#     print('not a file? ', fnb)
#     assert False

# if not Path(fnb2).is_file():
#     print('not a file? ', fnb2)
#     assert False

def get_one(fn):
    d = pickle.load(open(fn, 'rb'))
#     print(d.keys())
#     print('NTrials: ', len(d['trials']))
#     print(d['Params'].keys())
    return d

# d = get_one(fnb)
# # print(d['trials'][0]['inputSpikeTimes'])
# # print(d)

# for v in ['initIVStateFile', 'initANStateFile', 'simulationFilename', 'hocfile']:
#     d['Params'][v] = str(d['Params'][v])
# # print(d['Params'])
# # with(open(fnbo, 'wb')) as fh:
# #     pickle.dump(d, fh)



           Cell ID               ASA  nsyn(calculated)             delay           SRgroup            delay2        axonlength     branch length           syntype      postlocation


In [72]:
def make_sound(stimInfo):
    if isinstance(stimInfo['pip_start'], list):
        pips = stimInfo['pip_start']
    else:
        pips = [stimInfo['pip_start']]
    if stimInfo['soundtype'] == 'tonepip':
        stim = sound.TonePip(rate=stimInfo['Fs'], duration=stimInfo['run_duration'],
                          f0=stimInfo['F0'], dbspl=stimInfo['dB'],
                          ramp_duration=stimInfo['RF'], pip_duration=stimInfo['pip_duration'],
                          pip_start=pips)
    elif stimInfo['soundtype'] == 'SAM':
        stim = sound.SAMTone(rate=stimInfo['Fs'], duration=stimInfo['run_duration'],
                          f0=stimInfo['F0'], dbspl=stimInfo['dB'],
                          ramp_duration=stimInfo['RF'], 
                          fmod=stimInfo['fmod'], dmod=stimInfo['dmod'],
                          pip_duration=stimInfo['pip_duration'],
                          pip_start=pips)
    else:
        raise ValueError('StimInfo sound type %s not implemented' % stimInfo['soundtype'])

    stimWaveform = stim.generate()
    stimTimebase = stim.time
    return(stimTimebase, stimWaveform)

In [150]:
 def computeVS(data, andata=False, anf=0):
    allst = []
    if not andata:
        for trial in data['trials']:
            # print (trial)
            allst.extend(np.array(data['trials'][trial]['spikeTimes'])/1000.)
    else:
        for trial in data['trials']:
            for n in range(len(data['trials'][trial]['inputSpikeTimes'])):
                    allst.extend(np.array(data['trials'][trial]['inputSpikeTimes'][n])/1000.)
    allst = np.array(allst)
    allst = np.sort(allst) 
    si = data['trials'][0]['stimInfo']
    if si['soundtype'] == 'SAM':  # calculate vs and plot histogram
        # print('allst: ', allst)
        phasewin = [data['Params']['pip_start'][0] + 0.25*data['Params']['pip_duration'], 
            data['Params']['pip_start'][0] + data['Params']['pip_duration']]
        # print (phasewin)
        spkin = allst[np.where(allst > phasewin[0])]
        spikesinwin = spkin[np.where(spkin <= phasewin[1])]

        # set freq for VS calculation
        if si['soundtype'] == 'tone':
            f0 = data['Params']['F0']
            print ("Tone: f0=%.3f at %3.1f dbSPL, cell CF=%.3f" % 
                    (data['Params']['f0'], data['Params']['dB'], data['Params']['F0'] ))
        if si['soundtype'] == 'SAM':
            f0 = data['Params']['fmod']
            tstring = ("SAM Tone: f0=%.3f at %3.1f dbSPL, fMod=%3.1f  dMod=%5.2f, cell CF=%.3f" %
                 (data['Params']['F0'], data['Params']['dB'], data['Params']['fmod'], data['Params']['dmod'], data['Params']['F0']))
            print(tstring)
            # P.figure_handle.suptitle(tstring + '\n' + self.fn[self.Params['patterns'][0]].replace('_', '\_'), fontsize=9)
       # print('spikes: ', spikesinwin)
        print((' Sound type: ', si['soundtype']))
        if len(spikesinwin) < 10:
            vs = 0.
            print('AN Vector Strength: Insufficient spike count, n = %d' % (len(spikesinwin)))
        else:
            vs = vector_strength(spikesinwin*1000., f0)  # vs expects spikes in msec
            print('AN Vector Strength at %.1f: %7.3f, d=%.2f (us) Rayleigh: %7.3f  p = %.3e  n = %d' % (f0, vs['r'], vs['d']*1e6, vs['R'], vs['p'], vs['n']))
#             vspfile.add_data(self.Params['patterns'][0], self.Params['spirou'], vs['r'])
        return(vs)
    else:
        return 0

In [74]:
get_sims('VCN_c11', 'self')

Cell: 11: Soma mesh area: 1288.45  Soma hoc area: 883.41        Soma Inflation ratio: 1.458
fnb:  VCN_Cells/VCN_c11/Simulations/AN/AN_Result_VCN_c11_inp=self_XM13_nacncoop_II_soma=1.458_dend=1.458_ASA=1.458_delays_multisite_020_SAM_040dB_4000.0_400.0_50_MS.p
fnsl0:  [PosixPath('/Users/pbmanis/Desktop/Python/VCNModel/VCN_Cells/VCN_c11/Simulations/AN/AN_Result_VCN_c11_inp=self_XM13_nacncoop_II_soma=1.458_dend=1.458_ASA=1.458_delays_multisite_020_SAM_040dB_4000.0_200.0_100_MS.p'), PosixPath('/Users/pbmanis/Desktop/Python/VCNModel/VCN_Cells/VCN_c11/Simulations/AN/AN_Result_VCN_c11_inp=self_XM13_nacncoop_II_soma=1.458_dend=1.458_ASA=1.458_delays_multisite_020_SAM_040dB_4000.0_400.0_050_MS.p')]
name:  AN_Result_VCN_c11_inp=self_XM13_nacncoop_II_soma=1.458_dend=1.458_ASA=1.458_delays_multisite_020_SAM_040dB_4000.0_200.0_100_MS.p
name:  AN_Result_VCN_c11_inp=self_XM13_nacncoop_II_soma=1.458_dend=1.458_ASA=1.458_delays_multisite_020_SAM_040dB_4000.0_400.0_050_MS.p


[PosixPath('/Users/pbmanis/Desktop/Python/VCNModel/VCN_Cells/VCN_c11/Simulations/AN/AN_Result_VCN_c11_inp=self_XM13_nacncoop_II_soma=1.458_dend=1.458_ASA=1.458_delays_multisite_020_SAM_040dB_4000.0_200.0_100_MS.p'),
 PosixPath('/Users/pbmanis/Desktop/Python/VCNModel/VCN_Cells/VCN_c11/Simulations/AN/AN_Result_VCN_c11_inp=self_XM13_nacncoop_II_soma=1.458_dend=1.458_ASA=1.458_delays_multisite_020_SAM_040dB_4000.0_400.0_050_MS.p')]

In [154]:
def plot_SAM(P, ax, cell, inp, fnsl):
    # f, ax = mpl.subplots(3,1, figsize=(6,6))
    
#     P.figure_handle.suptitle(titlestring, fontsize=10)
    naxn = len(ax)
    print('fnsl: ', fnsl)
    for iax, fn in enumerate(fnsl):
        d = get_one(fn)
    #     print(d['trials'][0].keys())
        st = []
#         print(len(d['trials']))
        for i in list(d['trials'].keys()):
    #         for ansp in d['trials'][i]['spikeTimes']:
    #             print(ansp)
            st.extend(d['trials'][i]['spikeTimes'])
    #         print(d['trials'][i])
        st = np.array(st).ravel()
        ax[iax].hist(st, bins=np.arange(0, 250., 0.5))
        ax[iax].set_ylim((0,15))
#         PH.referenceline(ax[iax], 20)
        ax[iax].set_ylabel('spike count')

        PH.nice_plot(ax[iax])
        vs = computeVS(d)
        vsin = computeVS(d, andata=True)
#         txtlab = f"Input: {inp:s}  VS={vs['r']:.3f}, N spikes={vs['n']:d} R={vs['R']:7.2f} p={vs['p']:.3e}"
#         txtlab += f"  AN VS: {vsin['r']:.3f}"
        stimInfo = d['trials'][0]['stimInfo']
        f0 = stimInfo['F0']
        dbspl = stimInfo['dB']
        fmod = stimInfo['fmod']
        dmod = stimInfo['dmod']
#         print(cell)
#         print(inp)
#         print(dbspl)
#         print(fmod)
#         print(dmod)
        s=f"Cell:{cell:s} Input: {inp:s}  VS={vs['r']:.3f}\nN spikes={vs['n']:d} R={vs['R']:7.2f} p={vs['p']:.3e}"
        s += f"\nF: {float(f0):.1f}Hz {int(dbspl):3d}dB Fmod={fmod:.1f}Hz %mod={int(dmod):3d}"
        ax[iax].text(x=0.05, y=0.85,
                     s=s,
                     transform=ax[iax].transAxes, fontsize=8)
#         ax[iax].text(x=0.05, y=0.85,
#                      s=txtlab,
#                      transform=ax[iax].transAxes)
    t, w = make_sound(d['trials'][0]['stimInfo'])
    ax[2].plot(t*1000, w, linewidth=0.5)
    ax[2].set_xlabel('T (ms)')

    # get AN data from one fiber:
    ansp = []
    for trial in d['trials']:
                # print (trial)
        ansp.extend(np.array(d['trials'][trial]['inputSpikeTimes'][0])/1000.)
#         print(len(d['trials'][trial]['inputSpikeTimes'][0]))
    ax[1].hist(np.array(ansp)*1000., bins=np.arange(0, 250., 0.5))
    ax[1].set_ylabel('AN spike count')           

    PH.nice_plot(ax[iax])
    for i in range(0,naxn):
        ax[i].set_xlim(75, 225)
    #mpl.show()
    P.figure_handle.tight_layout()

#     mpl.savefig(outname)
#     mpl.show()                    

In [152]:
def plot_sim(cell, inp, fnsl):
    # PLOT FOR AN INPUT (one spike train)
    # f, ax = mpl.subplots(3,1, figsize=(8,6))
    P = PH.regular_grid(2, 1, figsize=(6,4),
                    verticalspacing=0.2, horizontalspacing=0.08,
                    margins={'leftmargin': 0.1, 'rightmargin': 0.1, 'topmargin': 0.1, 'bottommargin': 0.1})
    P.figure_handle.suptitle(titlestring, fontsize=10)
    ax = P.axarr.ravel()
    print('fnsl: ', fnsl)
    for iax, fn in enumerate(fnsl): #, fnb2]):
        d = get_one(fn)
#         print(d['trials'][0].keys())
        st = []
        for i in list(d['trials'].keys()):
            for ansp in d['trials'][i]['spikeTimes']:
#                 print('ansp: ', ansp)
                if isinstance(ansp, list):
                    st.extend(ansp)
                else:
                    st.append(ansp)
        st = np.array(st).ravel()
        ax[iax].hist(st, bins=np.arange(0, 250., 0.5))
        ax[iax].set_ylim((0,15))
#         PH.referenceline(ax[iax], 10)
        ax[iax].set_ylabel('spike count')

        PH.nice_plot(ax[iax])
        vs = computeVS(d)
        if iax == 0:
            ax[iax].text(x=0.05, y=0.85,
                     s=f"Cell:{cell:s} Input: {inp:s}  VS={vs['r']:.3f}, N spikes={vs['n']:d} R={vs['R']:7.2f} p={vs['p']:.3e}",
                     transform=ax[iax].transAxes, fontsize=9)
            
    t, w = make_sound(d['trials'][0]['stimInfo'])
    ax[1].plot(t*1000, w, linewidth=0.5)
    ax[1].set_xlabel('T (ms)')

    PH.nice_plot(ax[iax])
    for i in range(0,2):
        ax[i].set_xlim(75, 225)
    #mpl.show()
    P.figure_handle.tight_layout()



In [157]:
inps =  ['self']*4 # 'VCN_c13'
cells = ['VCN_c09', 'VCN_c11', 'VCN_c17', 'VCN_c18']
P = PH.regular_grid(3, len(cells), figsize=(12, 8),  order='columnsfirst',
                verticalspacing=0.05, horizontalspacing=0.08,
                margins={'leftmargin': 0.1, 'rightmargin': 0.1, 'topmargin': 0.1, 'bottommargin': 0.1})
ax = P.axarr.ravel()

# print(inps)
for i, cell in enumerate(cells):
#     print(cell, inps[i])
    fnsl = get_sims(cell, inps[i])
    ax0 = i*3
    plot_SAM(P, ax[ax0:ax0+3], cell, inps[i], fnsl)
mpl.show()


Cell: 09: Soma mesh area: 1340.36  Soma hoc area: 885.30        Soma Inflation ratio: 1.514
fnsl:  [PosixPath('/Users/pbmanis/Desktop/Python/VCNModel/VCN_Cells/VCN_c09/Simulations/AN/AN_Result_VCN_c09_inp=self_XM13_nacncoop_II_soma=1.514_dend=1.514_ASA=1.514_delays_multisite_020_SAM_040dB_4000.0_400.0_050_MS.p')]
SAM Tone: f0=4000.000 at 40.0 dbSPL, fMod=400.0  dMod=50.00, cell CF=4000.000
(' Sound type: ', 'SAM')
AN Vector Strength at 400.0:   0.142, d=521.13 (us) Rayleigh:  24.473  p = 3.074e-02  n = 172
SAM Tone: f0=4000.000 at 40.0 dbSPL, fMod=400.0  dMod=50.00, cell CF=4000.000
(' Sound type: ', 'SAM')
AN Vector Strength at 400.0:   0.092, d=536.32 (us) Rayleigh: 162.817  p = 3.349e-07  n = 1778


/Users/pbmanis/anaconda3/envs/py3mpl3/lib/python3.7/site-packages/ipykernel_launcher.py:63: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.


Cell: 11: Soma mesh area: 1288.45  Soma hoc area: 883.41        Soma Inflation ratio: 1.458
fnsl:  [PosixPath('/Users/pbmanis/Desktop/Python/VCNModel/VCN_Cells/VCN_c11/Simulations/AN/AN_Result_VCN_c11_inp=self_XM13_nacncoop_II_soma=1.458_dend=1.458_ASA=1.458_delays_multisite_020_SAM_040dB_4000.0_400.0_050_MS.p')]
SAM Tone: f0=4000.000 at 40.0 dbSPL, fMod=400.0  dMod=50.00, cell CF=4000.000
(' Sound type: ', 'SAM')
AN Vector Strength at 400.0:   0.147, d=519.73 (us) Rayleigh:  27.761  p = 1.695e-02  n = 189
SAM Tone: f0=4000.000 at 40.0 dbSPL, fMod=400.0  dMod=50.00, cell CF=4000.000
(' Sound type: ', 'SAM')
AN Vector Strength at 400.0:   0.096, d=534.93 (us) Rayleigh: 149.990  p = 5.357e-07  n = 1558
Cell: 17: Soma mesh area: 1357.62  Soma hoc area: 1009.83        Soma Inflation ratio: 1.344
fnsl:  [PosixPath('/Users/pbmanis/Desktop/Python/VCNModel/VCN_Cells/VCN_c17/Simulations/AN/AN_Result_VCN_c17_inp=self_XM13_nacncoop_II_soma=1.344_dend=1.344_ASA=1.344_delays_multisite_020_SAM_040dB

In [158]:
inps = ['VCN_c02', 'VCN_c05', 'VCN_c10', 'VCN_c13'] # ['self']*4 # 'VCN_c13'
cells = ['VCN_c17']*4 # ['VCN_c09', 'VCN_c11', 'VCN_c17', 'VCN_c18']
print(inps)
P = PH.regular_grid(3, len(cells), figsize=(12, 8),  order='columnsfirst',
                verticalspacing=0.05, horizontalspacing=0.08,
                margins={'leftmargin': 0.1, 'rightmargin': 0.1, 'topmargin': 0.1, 'bottommargin': 0.1})
ax = P.axarr.ravel()

for i, cell in enumerate(cells):
    print(cell, inps[i])
    fnsl = get_sims(cell, inps[i])
    ax0 = i*3
    plot_SAM(P, ax[ax0:ax0+3], cell, inps[i], fnsl)


['VCN_c02', 'VCN_c05', 'VCN_c10', 'VCN_c13']
VCN_c17 VCN_c02
Cell: 17: Soma mesh area: 1357.62  Soma hoc area: 1009.83        Soma Inflation ratio: 1.344
fnsl:  [PosixPath('/Users/pbmanis/Desktop/Python/VCNModel/VCN_Cells/VCN_c17/Simulations/AN/AN_Result_VCN_c17_inp=VCN_c02_XM13_nacncoop_II_soma=1.344_dend=1.344_ASA=1.344_delays_multisite_020_SAM_040dB_4000.0_400.0_050_MS.p')]
SAM Tone: f0=4000.000 at 40.0 dbSPL, fMod=400.0  dMod=50.00, cell CF=4000.000
(' Sound type: ', 'SAM')
AN Vector Strength at 400.0:   0.170, d=512.52 (us) Rayleigh:  20.447  p = 3.069e-02  n = 120
SAM Tone: f0=4000.000 at 40.0 dbSPL, fMod=400.0  dMod=50.00, cell CF=4000.000
(' Sound type: ', 'SAM')
AN Vector Strength at 400.0:   0.089, d=537.16 (us) Rayleigh: 117.734  p = 2.907e-05  n = 1327


/Users/pbmanis/anaconda3/envs/py3mpl3/lib/python3.7/site-packages/ipykernel_launcher.py:63: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.


VCN_c17 VCN_c05
Cell: 17: Soma mesh area: 1357.62  Soma hoc area: 1009.83        Soma Inflation ratio: 1.344
fnsl:  [PosixPath('/Users/pbmanis/Desktop/Python/VCNModel/VCN_Cells/VCN_c17/Simulations/AN/AN_Result_VCN_c17_inp=VCN_c05_XM13_nacncoop_II_soma=1.344_dend=1.344_ASA=1.344_delays_multisite_020_SAM_040dB_4000.0_400.0_050_MS.p')]
SAM Tone: f0=4000.000 at 40.0 dbSPL, fMod=400.0  dMod=50.00, cell CF=4000.000
(' Sound type: ', 'SAM')
AN Vector Strength at 400.0:   0.113, d=529.89 (us) Rayleigh:  12.792  p = 2.350e-01  n = 113
SAM Tone: f0=4000.000 at 40.0 dbSPL, fMod=400.0  dMod=50.00, cell CF=4000.000
(' Sound type: ', 'SAM')
AN Vector Strength at 400.0:   0.096, d=534.93 (us) Rayleigh: 149.990  p = 5.357e-07  n = 1558
VCN_c17 VCN_c10
Cell: 17: Soma mesh area: 1357.62  Soma hoc area: 1009.83        Soma Inflation ratio: 1.344
fnsl:  [PosixPath('/Users/pbmanis/Desktop/Python/VCNModel/VCN_Cells/VCN_c17/Simulations/AN/AN_Result_VCN_c17_inp=VCN_c10_XM13_nacncoop_II_soma=1.344_dend=1.344_A